<a href="https://colab.research.google.com/github/pythonwithzeeshan/learn-lessons/blob/main/LogisticRegression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Import Libraries**

In [19]:
# PyTorch aur numpy import
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd



**Dataset Creation**

In [30]:
df = pd.read_csv('diabetes_dataset.csv')
X = df[['BMI', 'Age']].values
y = df['Has_Diabetes'].values